In [1]:
## Package management

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

pd.options.display.float_format = '{:,.6f}'.format
np.set_printoptions(suppress=True)
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 4000

## SIS MODEL SIMULATION

In [2]:
def sis_model(G, gamma, beta, num_iterations, infected_percent):
    # Initialize the number of infected and susceptible nodes
    n = len(G)
    infected = np.random.binomial(1, infected_percent, size=n)
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_in_time = np.zeros((num_iterations, n), dtype=bool)
    adj_matrix_in_time[0,:] = infected

    # Iterate for the specified number of iterations
    for t in range(1, num_iterations):
        # Compute the transition probabilities for each node
        previous_state = adj_matrix_in_time[t-1,:]
        mask = np.matrix(np.split(np.random.binomial(1, beta, size=n*n), n))
        infected = np.dot(np.multiply(mask, adj_matrix), previous_state) > 0
        recovered = np.random.binomial(1, gamma, size=n) * previous_state
        adj_matrix_in_time[t,:] = (previous_state | infected) & ~recovered

    # Return the infected and susceptible lists
    infected_list = np.sum(adj_matrix_in_time, axis=1)
    susceptible_list = n - infected_list
    return infected_list, susceptible_list


## TASK 1  RESULTS

In [3]:
graphs_names = [
    "Complete",
    "Scale-Free", 
    "Small-World",
    "Star",
    "Regular Lattice",
    "Erdos-Renyi"
]
graphs = []
for name in graphs_names:
	if name == 'Complete':
		graphs.append(nx.complete_graph(1000))
	elif name == 'Scale-Free':
		graphs.append(nx.barabasi_albert_graph(1000, 3))
	elif name == 'Small-World':
		graphs.append(nx.watts_strogatz_graph(1000, 10, 0.5))
	elif name == 'Star':
		graphs.append(nx.star_graph(1000))
	elif name == 'Regular Lattice':
		graphs.append(nx.grid_graph(dim=[30, 30]))
	elif name == 'Erdos-Renyi':
		graphs.append(nx.erdos_renyi_graph(1000, 0.1))

In [4]:
def plot_sis_model(graphs_names,graphs, gamma, beta, num_iterations, infected_percent):
    
    for i, G in enumerate(graphs):
        # Simulate the spread of the disease on the current graph
        infected, susceptible = sis_model(G, gamma, beta, num_iterations, infected_percent)
        plt.figure()
        plt.plot([x / len(G) for x in infected], label='Infected')
        #plt.plot(susceptible, label='Susceptible')
        plt.title(f'SIS Model on {graphs_names[i]} Graph')
        plt.xlabel('Time step')
        plt.ylabel('Infected fraction')
        plt.legend()


In [ ]:
plot_sis_model(graphs_names, graphs, gamma=0.1, beta=0.2, num_iterations=300, infected_percent=0.3)

### TASK 2 

In [ ]:
# get eigen values
import pandas as pd

def get_eigenvalues(graphs_names, graphs):
    rows = []
    for i, G in enumerate(graphs):
        eigenvalues = nx.linalg.adjacency_spectrum(G)
        max_eigenvalue = max(eigenvalues)
        row = {'Graph': graphs_names[i], 'Eigenvalues': eigenvalues, 'Max Eigenvalue': max_eigenvalue}
        rows.append(row)
    
    df = pd.DataFrame(rows, columns=['Graph', 'Eigenvalues', 'Max Eigenvalue'])
    return df

df = get_eigenvalues(graphs_names, graphs)



In [ ]:
df

## Get threshold


In [ ]:
def find_threshold_beta(G, gamma):
    # Compute the maximum eigenvalue of the adjacency matrix
    max_eigenvalue = max(nx.linalg.adjacency_spectrum(G))
    # Compute the threshold
    beta = gamma / max_eigenvalue 
    return beta

In [ ]:
rows = []
for i, G in enumerate(graphs):
    
    df = pd.DataFrame(rows, columns=['Graph', 'Eigenvalues', 'Max Eigenvalue'])
    gamma = 0.1
    epsilon = 0.2
    # Compute the threshold
    beta_threshold = find_threshold_beta(G, gamma)
    beta_slightly_above = min(1,beta_threshold*(1+epsilon))
    beta_slightly_below = max(0,beta_threshold*(1-epsilon))
    # Simulate the SIS model with beta and gamma slightly above the threshold
    infected, susceptible = sis_model(G, gamma, beta_slightly_above, num_iterations=300, infected_percent=gamma)
	# Simulate the SIS model with beta and gamma slightly below the threshold
    infected2, susceptible2 = sis_model(G, gamma, beta_slightly_below, num_iterations=300, infected_percent=gamma)
    plt.figure()
    plt.plot([x/len(G) for x in infected], label='Infected (beta above threshhold)')
    plt.plot([x/len(G) for x in infected2], label='Infected (beta below threshhold)')
    plt.title(f'SIS Model on {graphs_names[i]} Graph')
    plt.xlabel('Time step')
    plt.ylabel('Fraction of Infected Population')
    plt.legend()
    
    row = {'Graph': graphs_names[i], 'gamma': gamma, 'beta_thresh': beta_threshold, 'beta_above': beta_slightly_above, 'beta_below': beta_slightly_below}
    rows.append(row)
 
df = pd.DataFrame(rows, columns=['Graph', 'gamma', 'beta_thresh','beta_above','beta_below'])
df